# 阅读理解模型

## Multi-Hop机制

### 优点

- 多次提取充足的信息

- 语义抽象化

### 方式

- 在句子级别Attention的layer推进

- 在TimeStep状态推进，与主序列维度一致


## Pointer network

### 基本概念

- 从输入序列中找到相应的tokens来作为输出，其利用Attention作为pointer,从输入序列中选择一个位置，并将这个位置所指向的词作为输出

- 个人理解：Pointer Network是将encoder层得到的注意力机制，作用于softmax函数，得到$alpha$概率分布，decoder层直接将其中概率最大的作为当前结果【之前的Seq2Seq的网络还会将$alpha$对各个输入元素做加权和】

### 基本原理

- 传统的注意力机制

![title](img/pn1.png)

- Pointer network的注意力机制

![title](img/pn2.png)


### 优点

- 提供了一种新视角去理解 Attention，把 Attention 作为一种求分布的手段。

- 对于输出字典长度不固定问题提供了一种新的解决方案。

- 将输入作为输出的一种补充手段，让输出部分可以更好的引入输入部分的信息。

### 论文地址

- https://arxiv.org/abs/1506.03134

- https://arxiv.org/abs/1608.07905

### 相关博文

- https://zhuanlan.zhihu.com/p/48959800

- https://lianhaimiao.github.io/2018/08/21/Pointer-Network-%E7%9A%84%E4%B8%80%E4%BA%9B%E7%90%86%E8%A7%A3/


## R-Net模型

### 架构图

![title](img/rnet1.png)

### 架构简介

- R-Net一共分为四层，由下至上，分别是编码层、交互层（其中交互层又分为文章问题交互层以及文章的自交互层）、输出层。

- 其中，编码层是一个Encoding过程，是双向RNN（这里采用GRU）,主要负责将输入的文章【Passage】和问题【Query】进行词向量转换

- 第二层是文章和问题的交互层，计算Q\P注意力机制

- 第三层是文章的自交互层，计算QP\P注意力机制

- 第四层是输出层，这里加入的Poniter Newtwork，来预测P(begin) & P(end)作为答案起始位置


### 基础原理

- 编码层 Question-Passage Encoding

> 最终每个词有一个词级别的向量，一个char级别的向量。

> 其中char级别的向量用来处理，例如 OOV、错误纠正等问题

> 之后通过一个双向RNN网络，得到文章和问题的向量表示

> - 这里的BiRNN使用GRU,代替了传统Attention中的LSTM,因为GRU少一个门，计算速度快

![title](img/rnet2.png)

- 交互层 Question-Passage Matching

> 将问题向量结果融合到文章中，从而得到问题向量下文章的语义理解

> 为了更好的提取和文章有关的问题信息，加入了门控机制$g_{t}$，主要是通过sigmoid函数输出概率分布，将相关度小的计算出相对小的概率

> 采用的是RNN网络

![title](img/rnet3.png)

![title](img/rnet4.png)

- 交互层  self-Matching

> 将上一层的输出结果做一个self-to-self的matching

> 此时所有的问题内容已经到融合到了文章中

> 这一层没有门控机制

> 采用BiRNN网络

![title](img/rnet5.png)

- 输出层 Answer Prediction

> 借鉴Pointer Network思想.最终输出的是问题的开始和结束位置

> 在这里融合的问题向量的加权重要性向量表示

![title](img/rnet6.png)

![title](img/rnet7.png)


### 特点

- 改进了RNN在机器阅读理解中的应用

- 将注意力机制融入RNN

- 门控循环神经网络

### 论文地址

- https://www.microsoft.com/en-us/research/wp-content/uploads/2017/05/r-net.pdf


### 相关博文

- https://zhuanlan.zhihu.com/p/36855204


## FusionNet

- 这个网络的提出贡献的两个重要的思想

> 单词历史

> 全关注注意力

### 基本概念

- 论文最大的价值是融合了当前主流的 Attention 模型的各个层次表示，并且一一作了介绍，可以通过论文了解当前主流 Attention 模型所用到的信息都是怎么得到的。

- 文章使用了 GloVe 表示单词，CoVe 表示上下文。

- 模型包含三个组成部分：输入向量、集成组件、融合过程

### 模型整体架构

![title](img/fusnet1.png)

- 整体结构包含三个组成部分

> 输入向量：上下文及问题中每个词的向量

> 集成组件：图中的长方形框，它是通过例如LSTM或者GRU的RNN结构实现的。

> 融合过程：图中的(1)(2)(2’)(3)(3’)，箭头汇入的地方代表融合


### 融合过程

- (1)词级融合: 单词是否出现

> 通过向上下文提供问题中的直接单词信息，我们可以快速放大上下文中更多相关区域。但是，如果一个单词根据上下文具有不同的语义含义，则可能没有帮助。 许多词级融合不是基于attention，例如有的将二进制特征附加到上下文单词，指示每个上下文单词是否出现在问题中。

> 提供了单词的直接信息，能够快速地聚焦关注的区域，例如简单地为每个单词附加一位，标注该单词是否出现在问题中。这种做法基本上是类似手工特征。

- (2)高层融合: 更新高层次表示

> 通过关于问题中的语义信息的上下文信息可以帮助我们找到正确的答案。但是高级信息比单词信息更不精确，这可能会导致模型对细节的了解较少。

> 比如，经过了LSTM的编码，这种特征再参与文章的判断（一般通过注意力机制）。但高层信息可能会丢失一部分细节。

- (2’)高层融合（可选择的）: 高层次融合到上下文单词

> 相似的，我们可以将Q的高层概念C的词级概念相融合。

> 和(1)差不多，但添加的是经过编码的高层信息，直接与Context的Embedding进行拼接。

- (3)自我加强融合 : 上下文高层次表示本身self-attention

> 由于上下文可能很长，并且文本的远距离部分可能相互依赖以充分理解内容，因此最近的进展已经提出将上下文融合到其自身中。

> 由于上下文包含过多的信息，因此一个常见的选择是在融合问题Q之后执行自增强融合。这使得我们能够更加了解与问题相关的区域。

> 信息与自身进行聚合（Transformer是一个典型例子），被认为这样可以对更长的信息进行关联。一般可以在(2)之后进行这一步骤，可以扩展关注问题(question-aware)的感受野。

- (3’) 自我加强融合（可选择的）: 融合前进行自我融合

> 另一种选择是直接调整问题Q上的自增效融合过程，然后我们可以在融合问题信息之前进行自增强融合。

> 与DCN提到的协同注意力机制和BiDAF中的机制一样，通过先对问题Q进行自增强融合，再将问题Q与文章进行高层融合。

### 两个重要思想

- 单词历史

> 作者对此提出的解释是，当人类阅读文章时，也会将文章中的各种元素抽象化，例如读到了“高山莱茵河”，会将其抽象为“一条欧洲的河”，更高层的抽象是“一条河”之类。但是，人类会将这些信息关联，而不会丢失底层的表示，因为这样会丢失细节。作者称原文加上各层级的抽象为“单词的历史记录”（History of Word，HoW），认为这种把各个层级的融合聚合在一起的方式是对HoW的一种“轻量级实现

> 一个单词从低层次到高层次所有表示向量一起称做这个词的history

> 方法: 将所有层拼接 

> 优点

> - 可以更好理解语义

> - 降低效率

- 全关注注意力

### 基本原理

- 编码层

> Glove: 300

> CoVe: 600

> POS: 12

> NER: 8

> Other: 2

> - C和Q中的每个单词都被转换成向量w（利用的是300维的CloVe 和 600维的CoVe向量）

- 交互层

> 交互层架构图

![title](img/fusnet2.png)

- 包含如下几部分

> 1. 单词注意力层

![title](img/fusnet3.png)

> 2. 阅读层

![title](img/fusnet4.png)

> 3. 问题理解层

![title](img/fusnet5.png)

> 4. 全关注互注意力层

![title](img/fusnet6.png)

> 5. 全关注自注意力层

![title](img/fusnet7.png)

- 输出层

![title](img/fusnet8.png)


### 模型优点

- 它提出了一种“history of word”的新概念，用于表征从最低的词向量到最高的语义层表示中的attention信息

- 它确定了更好地利用“history of word”概念的attention评分功能

- 它提出了一个全意识的多层次attention机制来捕获一个文本中的完整信息（如问题）并逐层将其用于对等体（如上下文或段落）中


### 论文地址

- https://arxiv.org/pdf/1711.07341.pdf


### 相关博文

- https://zhuanlan.zhihu.com/p/38959748

- https://blog.csdn.net/mottled233/article/details/104445417

## QA-Net

### 基本概念

- QANet 是在 BiDAF 的基础上进行改进的，主要的不同在 Embedding encoder layer 和 Model encoder layer。

> - Embedding encoder layer 由几个基本 block 堆叠而成，每个 block 的结构是：convolution-layer + self-attention-layer + feed-forward-layer。

> - Model encoder layer 和 BiDAF 差不多，不过这里用 CNN 而不是 RNN。


### 主要贡献

- 模型方面创新的用 CNN+attention 来完成阅读理解任务：在编码层放弃了 RNN，只采用 CNN 和 self-attention

- CNN 捕捉文本的局部结构信息（local interactions），self-attention 捕捉全局关系（global interactions），在没有牺牲准确率的情况下，加速了训练（训练速度提升了 3x-13x，预测速度提升 4x-9x）

- 数据增强方面通过神经翻译模型（把英语翻译成外语（德语/法语）再翻译回英语）的方式来扩充训练语料，增加文本多样性

### 创新点

- 卷积（可分离卷积）捕捉局部信息 （并行计算，加速）

- 卷积（可分离卷积）捕捉局部信息 （并行计算，加速）

- 数据增强

### 模型架构

![title](img/qanet1.png)

- 一个Encoder Block主要是如下几个模块（其中Transformer的EncoderBlock只有Attention和FFN，没有卷积）

> Positional Encoder

> 可分离卷积

> - https://plmsmile.github.io/2018/04/11/38-convolution/#xception

> Self-Attention

> - https://plmsmile.github.io/2018/08/29/48-attention-is-all-you-need/#self-attention

> 前向神经网络

### 基本原理

- Input Embedding Layer

> - Word embedding & Character embedding

> - 300 GLOVE embedding(OOV:\<UNK\> 随机)

> - 200 Character embedding

> - 基于 Bi-DAF

> - 采用两层 Highway Network

> - Character_max_len = 16

![title](img/qanet2.png)

- Embedding Encoder Layer

> 结构图

![title](img/qanet4.png)

> [convolution-layer × # + self-attention-layer + feed-forward-layer]

> Layer-Norm & Batch-Norm

> Depthwise separable convolution（记忆效果好，泛化能力强） size=7  d=128 num=4

> Multi-head Attention, a=8

> 输出d=128

![title](img/qanet3.png)

> - 各种注意力总结：https://plmsmile.github.io/2018/03/25/33-attention-summary/#%E9%94%AE%E5%80%BC%E5%AF%B9%E6%B3%A8%E6%84%8F%E5%8A%9B

- Attention Layer

![title](img/qanet5.png)

![title](img/qanet6.png)

- Model Encoder

> 与 Embedding Encoder Layer 一致

> Depthwise separable convolution：num=2

> 权值共享

![title](img/qanet7.png)


- Output Layer

![title](img/qanet8.png)


### 相关论文

- https://openreview.net/pdf?id=B14TlG-RW

- MRC框架：https://github.com/sogou/SogouMRCToolkit


### 相关博文

- https://zhuanlan.zhihu.com/p/56285539

- https://plmsmile.github.io/2018/08/30/49-qanet/